In [ ]:
import pandas as pd

In [5]:
!pip install pandas

twisted 18.7.0 requires PyHamcrest>=1.9.0, which is not installed.
You are using pip version 10.0.1, however version 24.0 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [ ]:
from selenium import webdriver
from selenium.webdriver import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import TimeoutException, NoSuchElementException, ElementClickInterceptedException
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import os

state_links = [
    "https://www.redbus.in/online-booking/ksrtc-kerala/?utm_source=rtchometile",
    "https://www.redbus.in/online-booking/apsrtc/?utm_source=rtchometile",
    "https://www.redbus.in/online-booking/tsrtc/?utm_source=rtchometile",
    "https://www.redbus.in/online-booking/ktcl/?utm_source=rtchometile",
    "https://www.redbus.in/online-booking/rsrtc/?utm_source=rtchometile",
    "https://www.redbus.in/online-booking/south-bengal-state-transport-corporation-sbstc/?utm_source=rtchometile",
    "https://www.redbus.in/online-booking/hrtc/?utm_source=rtchometile",
    "https://www.redbus.in/online-booking/astc/?utm_source=rtchometile",
    "https://www.redbus.in/online-booking/uttar-pradesh-state-road-transport-corporation-upsrtc/?utm_source=rtchometile",
    "https://www.redbus.in/online-booking/wbtc-ctc/?utm_source=rtchometile"
]

driver = webdriver.Chrome()
wait = WebDriverWait(driver, 20)
driver.maximize_window()

def extract_route_links(state_url):
    driver.get(state_url)
    time.sleep(3)
    route_links = []
    route_names = []
    for i in range(1, 6):
        route_elements = driver.find_elements(By.XPATH, "//a[@class='route']")
        for route in route_elements:
            route_links.append(route.get_attribute("href"))
            route_names.append(route.text)
        try:
            next_button_xpath = f'//div[@class="DC_117_pageTabs " and text()="{i+1}"]'
            next_button = driver.find_element(By.XPATH, next_button_xpath)
            if next_button:
                driver.execute_script("arguments[0].scrollIntoView(true);", next_button)
                time.sleep(2)
                next_button.click()
                time.sleep(3)
            else:
                break
        except (NoSuchElementException, ElementClickInterceptedException):
            break
    return route_links, route_names

def extract_state_data(state_links):
    Bus_names = []
    Bus_types = []
    Start_Time = []
    End_Time = []
    Ratings = []
    Total_Duration = []
    Prices = []
    Seats_Available = []
    Route_names = []
    Route_links = []
    State_names = []

    driver_k = webdriver.Chrome()

    for state_index, state_url in enumerate(state_links):
        route_links, route_names = extract_route_links(state_url)
        state_name = state_url.split('/')[-2]
        df_routes = pd.DataFrame({"Route_name": route_names, "Route_link": route_links})

        for i, r in df_routes.iterrows():
            link = r["Route_link"]
            route_name = r["Route_name"]
            driver_k.get(link)
            time.sleep(3)
            scrolling = True
            while scrolling:
                old_page_source = driver_k.page_source
                ActionChains(driver_k).send_keys(Keys.PAGE_DOWN).perform()
                time.sleep(5)
                new_page_source = driver_k.page_source
                if new_page_source == old_page_source:
                    scrolling = False

            try:
                bus_names = driver_k.find_elements(By.XPATH, "//div[@class='travels lh-24 f-bold d-color']")
                bus_types = driver_k.find_elements(By.XPATH, "//div[@class='bus-type f-12 m-top-16 l-color evBus']")
                start_times = driver_k.find_elements(By.XPATH, "//*[@class='dp-time f-19 d-color f-bold']")
                end_times = driver_k.find_elements(By.XPATH, "//*[@class='bp-time f-19 d-color disp-Inline']")
                total_durations = driver_k.find_elements(By.XPATH, "//*[@class='dur l-color lh-24']")
                ratings = driver_k.find_elements(By.XPATH, "//div[@class='clearfix row-one']/div[@class='column-six p-right-10 w-10 fl']")
                prices = driver_k.find_elements(By.XPATH, "//*[@class='fare d-block']")
                seats = driver_k.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")

                for bus in bus_names:
                    Bus_names.append(bus.text)
                    Route_names.append(route_name)
                    Route_links.append(link)
                    State_names.append(state_name)

                for bus_type in bus_types:
                    Bus_types.append(bus_type.text)

                for start_time in start_times:
                    Start_Time.append(start_time.text)

                for end_time in end_times:
                    End_Time.append(end_time.text)

                for total_duration in total_durations:
                    Total_Duration.append(total_duration.text)

                for rating in ratings:
                    Ratings.append(rating.text)

                for price in prices:
                    Prices.append(price.text)

                for seat in seats:
                    Seats_Available.append(seat.text)

            except NoSuchElementException as e:
                continue

    bus_data_all_states = {
        'State_name': State_names,
        'Bus_name': Bus_names,
        'Bus_type': Bus_types,
        'Start_time': Start_Time,
        'End_time': End_Time,
        'Total_duration': Total_Duration,
        'Price': Prices,
        'Seats_Available': Seats_Available,
        'Ratings': Ratings,
        'Route_link': Route_links,
        'Route_name': Route_names
    }

    df_bus_all_states = pd.DataFrame(bus_data_all_states)
    output_folder = 'scraped_data'
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    csv_file_path = os.path.join(output_folder, 'red_bus_data_all_states.csv')
    df_bus_all_states.to_csv(csv_file_path, index=False)

    driver_k.quit()

extract_state_data(state_links)
driver.quit()


In [ ]:
#Output:
#No more pages at step 2
#No more pages at step 5
#No more pages at step 3
#No more pages at step 4
#No more pages at step 2
#No more pages at step 5
#No more pages at step 4
#No more pages at step 5
#No more pages at step 5
#No more pages at step 4
#Data for all states saved to scraped_data\red_bus_data_all_states.csv successfully.


In [ ]:
import pymysql
import pandas as pd
import re

db_config = {
    'host': '127.0.0.1',
    'user': 'root',
    'password': 'Mithun252001@',
    'database': 'bus_data',
    'charset': 'utf8mb4',
}

connection = pymysql.connect(
    host=db_config['host'],
    user=db_config['user'],
    password=db_config['password'],
    charset=db_config['charset']
)

cursor = connection.cursor()

cursor.execute("CREATE DATABASE IF NOT EXISTS bus_data")
cursor.execute("USE bus_data")

create_table_query = """
CREATE TABLE IF NOT EXISTS bus_routes (
    id INT AUTO_INCREMENT PRIMARY KEY,
    route_name TEXT,
    route_link TEXT,
    busname TEXT, 
    bustype TEXT,
    departing_time TIME,
    duration TEXT,
    reaching_time TIME,
    star_rating FLOAT,
    price DECIMAL(10, 2),
    seats_available INT
);
"""
cursor.execute(create_table_query)

connection.commit()

def extract_rating(rating_str):
    try:
        rating_str = rating_str.split('\n')[0].strip()
        return float(rating_str) if rating_str else None
    except ValueError:
        return None

def clean_price(price_str):
    try:
        numeric_part = re.sub(r'[^\d.,]', '', price_str)
        numeric_part = numeric_part.replace(',', '')
        return float(numeric_part) if numeric_part else None
    except ValueError:
        return None

def clean_seats(seats_str):
    try:
        numeric_part = re.sub(r'[^\d]', '', seats_str)
        return int(numeric_part) if numeric_part else None
    except ValueError:
        return None

def insert_data_to_db(df):
    insert_query = """
    INSERT INTO bus_routes (
        route_name, route_link, busname, bustype, departing_time, 
        duration, reaching_time, star_rating, price, seats_available
    ) 
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """

    for index, row in df.iterrows():
        cursor.execute(insert_query, (
            row['Route_name'], 
            row['Route_link'], 
            row['Bus_name'], 
            row['Bus_type'],
            row['Start_time'],  
            row['Total_duration'], 
            row['End_time'],  
            extract_rating(row['Ratings']),  
            clean_price(row['Price']),  
            clean_seats(row['Seats_Available'])  
        ))

    connection.commit()

csv_file_path = 'scraped_data/red_bus_data_all_states.csv'
df_scraped = pd.read_csv(csv_file_path)

insert_data_to_db(df_scraped)

cursor.close()
connection.close()


In [ ]:
#Output:
#Null values in Price column: 0
#Null values in Seats_Available column: 0
#Data inserted into MySQL database successfully.


In [ ]:
import streamlit as st
import pandas as pd
import pymysql

db_config = {
    'host': '127.0.0.1',
    'user': 'root',
    'password': 'Mithun252001@',
    'database': 'bus_data',
    'charset': 'utf8mb4',
}

def fetch_data_from_db(query):
    connection = pymysql.connect(
        host=db_config['host'],
        user=db_config['user'],
        password=db_config['password'],
        database=db_config['database'],
        charset=db_config['charset']
    )
    df = pd.read_sql(query, connection)
    connection.close()
    return df

st.title('Bus Booking Data - Filter and Analysis')

with st.sidebar:
    st.header("Filter Options")
    route = st.selectbox("Select the Route", ["All Routes"] + fetch_data_from_db("SELECT DISTINCT route_name FROM bus_routes")['route_name'].tolist())
    bustype = st.selectbox("Select the Seat Type", ["All Types", "Sleeper", "Seater", "Non A/C", "A/C"])
    star_rating = st.selectbox("Select the Ratings", ["All Ratings", "1 to 2", "2 to 3", "3 to 4", "4 to 5"])
    time_filter = st.selectbox("Starting time", ["All", "06:00 - 12:00", "12:00 - 18:00", "18:00 - 00:00", "00:00 - 06:00"])
    price_range = st.selectbox("Bus Fare Range", ["All", "0 - 500", "500 - 1000", "1000 - 1500", "1500+"])
    seat_availability = st.selectbox("Seats Availability", ["All", "0-5", "5-10", "10-20", "20+"])

query = "SELECT * FROM bus_routes WHERE 1=1"

if route != "All Routes":
    query += f" AND route_name = '{route}'"

if bustype != "All Types":
    query += f" AND bustype LIKE '%{bustype}%'"

if star_rating != "All Ratings":
    rating_min, rating_max = map(float, star_rating.split(' to '))
    query += f" AND star_rating BETWEEN {rating_min} AND {rating_max}"

if time_filter != "All":
    time_ranges = {
        "06:00 - 12:00": ("06:00:00", "12:00:00"),
        "12:00 - 18:00": ("12:00:00", "18:00:00"),
        "18:00 - 00:00": ("18:00:00", "23:59:59"),
        "00:00 - 06:00": ("00:00:00", "06:00:00")
    }
    time_min, time_max = time_ranges[time_filter]
    query += f" AND departing_time BETWEEN '{time_min}' AND '{time_max}'"

if price_range != "All":
    price_min, price_max = map(int, price_range.split(' - ')) if ' - ' in price_range else (1500, 10000)
    query += f" AND price BETWEEN {price_min} AND {price_max}"

if seat_availability != "All":
    seat_min, seat_max = map(int, seat_availability.split('-')) if '-' in seat_availability else (20, 100)
    query += f" AND seats_available BETWEEN {seat_min} AND {seat_max}"

df = fetch_data_from_db(query)

if 'route_link' in df.columns:
    df = df.drop(columns=['route_link'])

if not df.empty:
    st.dataframe(df)
else:
    st.write("No results found for the selected filters.")

st.subheader("Data Analysis")

if not df.empty:
    st.write(f"**Average Price**: ₹{df['price'].mean():.2f}")
    st.write(f"**Average Star Rating**: {df['star_rating'].mean():.2f}")
    st.write(f"**Total Buses Available**: {df.shape[0]}")
else:
    st.write("No data to analyze")


In [ ]:
#Output: Displayed in streamlit.